In [2]:
import torch
import numpy as np
import scipy.io as sio
import os

In [3]:
if torch.cuda.is_available():
   device = torch.device("cuda")
   print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [47]:
# Paramètres du modèle
N = 2 # dim de l'input x
m = 10 # dim de l'output y
T = 1 # borne sup pour x
nb_points = 10 
nb_maps = 100 # nombre d'images que l'on va générer
nb_blocks = 2 # nombre de blocs dans le modèle
M = 10 # nombre d'échantillons pour l'approximation de Monte Carlo

In [89]:
omega = torch.tensor([[0, 1] for _ in range(N)])
X = torch.zeros((nb_maps, nb_points, N))

for _ in range(nb_maps):
    x = torch.rand(nb_points, N) * (omega[:, 1] - omega[:, 0]) + omega[:, 0] # all the coordinates of x are between 0 and 1
    X[_, :, :] = x
X = torch.Tensor(np.expand_dims(X, axis=1))
print(f'La dimension de x est {x.size()}')

tensor([[[0.3993, 0.7746],
         [0.8363, 0.7717],
         [0.6391, 0.3520],
         ...,
         [0.8841, 0.8166],
         [0.7123, 0.7780],
         [0.0135, 0.0353]],

        [[0.8790, 0.9101],
         [0.1632, 0.4167],
         [0.4842, 0.3484],
         ...,
         [0.4323, 0.9838],
         [0.6947, 0.9691],
         [0.8011, 0.9819]],

        [[0.8550, 0.1772],
         [0.8089, 0.7241],
         [0.2256, 0.5109],
         ...,
         [0.3120, 0.8858],
         [0.8807, 0.2471],
         [0.3191, 0.8327]],

        ...,

        [[0.5937, 0.7349],
         [0.8617, 0.3459],
         [0.9040, 0.2119],
         ...,
         [0.3611, 0.2821],
         [0.4607, 0.2944],
         [0.9514, 0.7599]],

        [[0.1060, 0.0364],
         [0.8852, 0.4874],
         [0.5199, 0.2959],
         ...,
         [0.9905, 0.3666],
         [0.2988, 0.6709],
         [0.4042, 0.8248]],

        [[0.6239, 0.9388],
         [0.8745, 0.9800],
         [0.8333, 0.9794],
         ...,
 

In [6]:
class Block(torch.nn.Module):
    def __init__(self, m) -> None:
        super(Block, self).__init__()
        self.fc = torch.nn.Linear(m, m)
        self.activation = torch.nn.Tanh()

    def forward(self, x):
        identity = x
        x = self.fc(x)
        x = self.activation(x)
        x = self.fc(x)
        x = self.activation(x)
        return x + identity
    
class Model(torch.nn.Module):
    def __init__(self, N, m, nb_blocks) -> None:
        super(Model, self).__init__()

    def forward(self, x):
        x = torch.nn.Linear(N, m)(x)
        for _ in range(nb_blocks):
            x = Block(10)(x)
        return x

In [38]:
def conv(x : torch.Tensor, dx = True, delta : float = 1/nb_points) -> torch.Tensor:
    if dx:
        convx = torch.zeros(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if j == 0 :
                    convx[i, j] = (x[i, j+1] - x[i, j])/delta # forward 
                elif j == x.shape[1] - 1:
                    convx[i, j] = (x[i, j] - x[i, j-1])/delta # backward
                else:
                    convx[i, j] = (x[i, j+1] - x[i, j-1])/(2*delta) # central
        return convx
    else:
        convy = torch.zeros(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if i == 0 :
                    convy[i, j] = (x[i+1, j] - x[i, j])/delta # forward
                elif i == x.shape[0] - 1:
                    convy[i, j] = (x[i, j] - x[i-1, j])/delta # backward
                else:
                    convy[i, j] = (x[i+1, j] - x[i-1, j])/(2*delta) # central
        return convy

In [100]:
# Code simple pour une seule image
gen = Model(N, m, nb_blocks)

# Monte Carlo
for _ in range(10) :
    output = gen(x)
    output_dx, output_dy = conv(output), conv(output, dx = False)
    L1, L2 = torch.pow(output_dx, 2), torch.pow(output_dy, 2)

In [96]:
gen = Model(N, m, nb_blocks)
output = gen(X)
output_dX, output_dY = torch.zeros(output.shape), torch.zeros(output.shape)

for i, out in enumerate(output) :
    output_dx, output_dy = conv(out[0]), conv(out[0], dx = False)
    output_dx, output_dy = output_dx.expand(1, 1, -1, -1), output_dy.expand(1, 1, -1, -1)
    output_dX[i, :, :, :], output_dY[i, :, :, :] = output_dx, output_dy
    
L1, L2 = torch.pow(output_dX, 2)/2, torch.pow(output_dY, 2)/2
# gen_opt = torch.optim.Adam(gen.parameters(), lr=1e-4)

torch.Size([100, 1, 10, 10])


In [97]:
for _ in range(10):
    output = gen(x)
    print(output.sum())

tensor(12.3533, grad_fn=<SumBackward0>)
tensor(-5.2215, grad_fn=<SumBackward0>)
tensor(17.6109, grad_fn=<SumBackward0>)
tensor(2.1467, grad_fn=<SumBackward0>)
tensor(29.6196, grad_fn=<SumBackward0>)
tensor(34.3030, grad_fn=<SumBackward0>)
tensor(2.3834, grad_fn=<SumBackward0>)
tensor(-25.3725, grad_fn=<SumBackward0>)
tensor(-34.0533, grad_fn=<SumBackward0>)
tensor(-23.9935, grad_fn=<SumBackward0>)
